In [1]:
KotlinVersion.CURRENT             // 코틀린 현재 버전 

1.8.0

In [2]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.6.4")

In [3]:
:classpath

Current classpath (16 paths):
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/lib-0.11.0-170.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/api-0.11.0-170.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-script-runtime-1.8.0-dev-3517.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-reflect-1.7.10.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-1.7.10.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-common-1.7.10.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/annotations-13.0.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlinx-serialization-json-jvm-1.3.3.jar
/Users/a06

In [4]:
import kotlinx.coroutines.*
import kotlinx.coroutines.channels.*

# 2 예외처리 

In [18]:
fun main() = runBlocking {                            // 런블러킹 스코프 구성 
    val job = launch {                                  // 자식 코루틴 
        try {
                repeat(1000) { i ->                     // 1000 개의 작업 
                println("코루틴 job 실행 : $i ...")
                delay(500L)                             // 일시중단 
                }
        } catch(e : Exception) {                        // 작업 종료로 예외 발생 
                println("중단에 따른 예외 :" + e.message)   // 예외처리 
        }
    }  
    delay(1300L)                                        // 런블러킹 스코프에서 지연
    println("메인 처리후 자식 종료")                         //
    job.cancel()                                        // 현재 코루틴 작업 종료
    job.join()                                         // 코루틴 작업 종료
    println("메인 종료 .")
}

main()

코루틴 job 실행 : 0 ...
코루틴 job 실행 : 1 ...
코루틴 job 실행 : 2 ...
메인 처리후 자식 종료
중단에 따른 예외 :StandaloneCoroutine was cancelled
메인 종료 .


## 예외핸들러 정의 및 전달 

In [19]:
fun main() {
    val handler = CoroutineExceptionHandler {                     // 예외처리 핸들러 작성 
             _, exception -> println("예외처리 : $exception")}
    try {
        runBlocking {                                             // 런블러킹 스코프 구성   
            val job = launch(handler) {                           // 자식 코루틴 
                println("코루틴 실행 ")
                delay(500L)                                       // 일시중단 
                throw Exception("첫번째 코루틴 내에서 예외 발생 ")
            }
            delay(1300L)                                          // 런블러킹 스코프에서 지연
            println("메인 처리후 자식 종료")                           //
            println("메인 종료 .")
        }
    } catch (e : Exception) {
        println(e.message)
    }
}

main()

코루틴 실행 
첫번째 코루틴 내에서 예외 발생 


In [ ]:
## 수퍼바이저 처리 

In [20]:
fun main() = runBlocking {

    try {  
        supervisorScope {                                        // 수퍼바이저스코프로 예외 처리
            val job = async  {                                   // 어싱크에서 예외가 발생하면 부모에게 전달 자러
                println("코루틴 실행 ")
                delay(500L)                                       // 일시중단 
                throw Exception("첫번째 코루틴 내에서 예외 발생 ")       // 예외발생 
            }
                                                
            println("메인 처리후 자식 종료")                           // 
            try {                                                 // 부모에게 예외를 던져서 실제 처리가 안됨 
                job.await()
            } catch(e:Exception) {                                // 수퍼바이저스코프로 처리해야 예외전파가 중단됨
                println("예외를 다시 전달")                           // 예외를 다시 전달해서 출력됨
                println(e.message)
            }
            println("메인 종료 .")
        }
    } catch (e : Exception) {                                     // 수퍼바이저스코프 내에서 예외처리해서 
        println("부모영역까지 예외전달 ")                               // 예외처리하지 않믐
        println(e.message)
    }
}

main()

메인 처리후 자식 종료
코루틴 실행 
예외를 다시 전달
첫번째 코루틴 내에서 예외 발생 
메인 종료 .


## 수퍼바이저 스코프로 예외처리

In [21]:
// suspend fun <R> supervisorScope(                  // 수퍼바이저스코프도 일시중단함수
// block: suspend CoroutineScope.() -> R): R         // 매개변수로 람다표현식 받음 

fun main31() = runBlocking {                          // 최상위 부모 코루틴
    try { 
        supervisorScope {                             // 수퍼바이저 스코프로 처리 
            val firstchild = launch {                 // 첫번째 자식 코루틴 
                try { 
                    println("잠시 중단")                // 실행
                    delay(500) 
                    println("다시 시작")                // 예외발생으로 미실행 
                }
                finally { 
                    println("자식 코루틴 중단처리 ")       // 예외을 전달을 받아서 처리됨 
                } }  
            val secondchild = launch {                // 두번째 자식 코루틴 
                println("잠시 중단 2")                  // 실행 
                delay(1000) 
                println("다시 시작 2")  }               // 예외 발생으로 미실행
            yield() 
            println("예외 발생 시킴 ")                   // 수퍼바이저스코프 자식 코루틴에게만 예외 전달
            throw AssertionError("자식들에게 전파")       // 예외 발생 
        } 
    } 
    catch(e: AssertionError) { 
        println("예외 갭처  , ${e.message}") 
    } 
    val job = GlobalScope.launch {                    // 수퍼바이저스코프와 형제 코루틴 
        println("잠시 중단 3")                          // 에외와 상관없이 처리
        delay(1000) 
        println("다시 시작 3 ")
    }
    delay(2000)
}
main31()

잠시 중단
잠시 중단 2
예외 발생 시킴 
자식 코루틴 중단처리 
예외 갭처  , 자식들에게 전파
잠시 중단 3
다시 시작 3 


## 수퍼바이저 잡 처리

In [22]:
// fun SupervisorJob(parent: Job? = null)                            // 수퍼바이저잡 함수 
// : CompletableJob                                                  // 잡을 반환
// interface CompletableJob : Job                                    // 컴플리터블잡 인터페이스 
// interface Job : CoroutineContext.Element                          // 잡도 코루틴 컨텍스트의 엘리먼트
fun main32() = runBlocking { 
    val supervisor = SupervisorJob()                                 // 수퍼바이저 잡 지정 
    val handler = CoroutineExceptionHandler {                        // 예외처리 핸들러 작성 
             _, exception -> println("예외처리 : $exception")}
    try { 
        with(CoroutineScope(coroutineContext 
                            + supervisor)) { 
            val firstChild = launch(handler) {                        // 코루틴에 예외처리 전달 
                println("첫번째 코루틴 실행 ") 
                throw AssertionError("첫번째 코루틴 내에서 예외 발생 ")      // 예외발생 
            } 
            val secondChild = launch {     
                firstChild.join()                                     // 첫번째 코루틴 조인 
                println("첫번째 코루틴 상태: ${firstChild.isCancelled}" ) // 첫번째 코루틴 상태 
                try { 
                    delay(Long.MAX_VALUE) 
                } 
                finally {                                              // 중단해서 반드시 처리  
                    println("두번째 코루틴 처리 ") 
                } 
            }                                                          
            firstChild.join()                                          // 첫번째 코루틴 중
            println("수퍼바이저로 중단 처리 ") 
            supervisor.cancel()                                        // 수퍼바이저 중단 
            secondChild.join()                                         // 두번째 코루틴 중단  
        }
    } catch (e: CancellationException) { 
             println("전체 코루틴 완료 !") 
    } 
}
main32()

첫번째 코루틴 실행 
예외처리 : java.lang.AssertionError: 첫번째 코루틴 내에서 예외 발생 
첫번째 코루틴 상태: true
수퍼바이저로 중단 처리 
두번째 코루틴 처리 
